In [ ]:
import numpy as np
import random
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, OPTICS, MeanShift ,SpectralClustering
from sklearn_extra.cluster import KMedoids
from sklearn.mixture import GaussianMixture
import pandas as pd
from sklearn.utils import shuffle

import os
import torch
import hdbscan
# class IncrementalSampling(object):
#     def __init__(self,prototype_nums = 10,cluster_strategy = 'kmeans'):
#         self.incremental_data_map = {}
#         self.data  = np.array([])
#         self.label = np.array([])
#         self.k_value = prototype_nums
#         # 2024 06 07
#         self.cluster_strategy = cluster_strategy
#     def split_current_data_by_class(self):
#         """把当前轮的数据按类别划分"""
#         # 获取唯一的类别标签
#         current_unique_labels = np.unique(self.label)
#         # 按类别分割当前轮获得的数据
#         data_by_class = {}  # map {"label":[the data belong to the label]}
#         for current_data_label in current_unique_labels:
#             indices = np.where(self.label == current_data_label)[0]
#             data_by_class[current_data_label] = self.data[indices]
#         return data_by_class, current_unique_labels
#     def data_combined(self):
#         """把当前轮的数据，和类map中的数据进行合并"""
#         current_data_map_by_class, current_unique_labels = self.split_current_data_by_class()
#         # 考虑到有新类出现的情况
#         # 如果current_unique_labels有新label,直接扩充incremental_data_map
#         # 如果label 是incremental_data_map中已经有的，扩张incremental_data_map对应label中data的长度
#         for new_data_label in current_unique_labels:
#             if new_data_label in self.incremental_data_map:
#                 # 增量map中已经有这个标签的数据了，那就扩充这个数据
#                 self.incremental_data_map[new_data_label] = np.concatenate(
#                     (self.incremental_data_map[new_data_label], current_data_map_by_class[new_data_label])
#                 )
#             else:
#                 # 如果增量map中没有这个标签的的数据，就扩充增量map
#                 self.incremental_data_map[new_data_label] = current_data_map_by_class[new_data_label]
#     def cluster_data(self,data, num_clusters):
#         # ['kmeans','spectral','kmeans++','kmedoids','OPTICS','meanshift','gmm']
#         # kmeans = KMeans(n_clusters=num_clusters,n_init='auto')
#         cluster_centers = np.ndarray([])
#         if self.cluster_strategy == 'kmeans':
#             kmeans = KMeans(n_clusters=num_clusters)
#             kmeans.fit(data)
#             cluster_centers = kmeans.cluster_centers_
#         elif self.cluster_strategy == 'spectral':
#             spectral = SpectralClustering(n_clusters=num_clusters, random_state=0, affinity='nearest_neighbors').fit(data)
#             labels_spectral = spectral.labels_
#             cluster_centers = np.array([data[labels_spectral == i].mean(axis=0) for i in range(num_clusters)])
#         elif self.cluster_strategy.lower() == 'hdbscan':
#             min_cluster_size = 2
#             hdb = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size).fit(data)
#             labels_hdbscan = hdb.labels_
#             cluster_centers = np.array([data[labels_hdbscan == i].mean(axis=0) for i in range(len(set(labels_hdbscan)) - (1 if -1 in labels_hdbscan else 0))])
#         elif self.cluster_strategy.lower() == 'kmeans++' or self.cluster_strategy.lower() == 'kmeansplusplus' :
#             kmeans_plus_plus = KMeans(n_clusters=num_clusters, init='k-means++', random_state=0).fit(data)
#             cluster_centers = kmeans_plus_plus.cluster_centers_
#         elif self.cluster_strategy.lower() =='kmedoids':
#             kmedoids = KMedoids(n_clusters=num_clusters, random_state=0).fit(data)
#             cluster_centers = kmedoids.cluster_centers_
#         elif self.cluster_strategy.upper() =='OPTICS':
#             min_samples = 2
#             optics = OPTICS(min_samples=min_samples).fit(data)
#             labels_optics = optics.labels_
#             cluster_centers = np.array([data[labels_optics == i].mean(axis=0) for i in range(len(set(labels_optics)) - (1 if -1 in labels_optics else 0))])
#         elif self.cluster_strategy.lower() =='meanshift':
#             bandwidth = 0.1
#             mean_shift = MeanShift(bandwidth=bandwidth).fit(data)
#             cluster_centers = mean_shift.cluster_centers_
#         elif self.cluster_strategy.lower() =='gmm':
#             gmm = GaussianMixture(n_components=num_clusters, random_state=0).fit(data)
#             cluster_centers = gmm.means_  # 高斯混合模型的质心是每个成分的均值
#         return  cluster_centers
#     def reCludter(self):
#         # incremental_data_map的大小进行压缩self.reCluster_instance_nums
#         new_cluster = {}
#         for label, data in self.incremental_data_map.items():
#             if len(data) <= self.k_value:
#                 # 全部保存
#                 new_cluster[label] = data
#             else:
#                 sampled_nums = self.k_value
#                 clusters = self.cluster_data(data = data, num_clusters=sampled_nums)
#                 new_cluster[label] = clusters
#         self.incremental_data_map = new_cluster
#     def compute_sampling_nums(self,combinde_map):
#         # initialize
#         min_length = float('inf')  # 初始化最小长度为正无穷大
#         max_length = 0  # 初始化最大长度为0
#         for label, data in combinde_map.items():
#             data_length = len(data)
#             if data_length < min_length:
#                 min_length = data_length
#             if data_length > max_length:
#                 max_length = data_length
#         return min_length, max_length
#     def data_sampling(self,sampling_nums):
#         resampling_data = []
#         resampling_label = []
#         for label, data in self.incremental_data_map.items():
#             if len(data) > sampling_nums:
#                 # 那就是下采样了,不放回采样
#                 sampled_indices = random.sample(range(len(data)), sampling_nums)
#                 sampled_data = [data[i] for i in sampled_indices]
#                 resampling_data.extend(sampled_data)
#                 resampling_label.extend([label] * sampling_nums)
#             elif len(data) == sampling_nums:
#                 # 直接复制
#                 resampling_data.extend(data)
#                 resampling_label.extend([label] * sampling_nums)
#             else:
#                 # 上采样,保存原样本
#                 resampling_data.extend(data)
#                 # 随机有放回的找差额部分
#                 sampled_data = random.choices(data, k=(sampling_nums-len(data)))
#                 resampling_data.extend(sampled_data)
#                 resampling_label.extend([label] * sampling_nums)
#         # 洗牌
#         combined_data = list(zip(resampling_data, resampling_label))
#         random.shuffle(combined_data)
#         resampling_data, resampling_label = zip(*combined_data)
#         return resampling_data,resampling_label
#     def fit(self,incremental_prototype_map,data,label,sampling_strategy = "OverSampling"):
#         self.incremental_data_map = incremental_prototype_map
#         self.data = data
#         self.label = label
#         self.data_combined()
#         resampling_data = []
#         resampling_label = []
#         min_length, max_length =self.compute_sampling_nums(self.incremental_data_map)
#         if sampling_strategy.lower() == "oversampling":
#             resampling_data,resampling_label = self.data_sampling(max_length)
#         elif sampling_strategy.lower() == "downsampling":
#             resampling_data,resampling_label = self.data_sampling(min_length)
#         else:
#             print("No sampling measures have been taken")
#         self.reCludter()
#         return resampling_data,resampling_label,self.incremental_data_map


In [32]:
# 上诉代码没有共享模型
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,recall_score, accuracy_score, roc_auc_score,cohen_kappa_score,confusion_matrix,precision_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import copy
import os
import random
from sklearn.cluster import KMeans
from numpy.linalg import norm
from sklearn.neighbors import NearestNeighbors
import numpy as np
import random
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, OPTICS, MeanShift ,SpectralClustering
from sklearn_extra.cluster import KMedoids
from sklearn.mixture import GaussianMixture
import pandas as pd
from sklearn.utils import shuffle

import os
import torch
import hdbscan
import warnings
# 定义MLP模型
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class IncrementalSampling(object):
    def __init__(self,prototype_nums = 30,cluster_strategy = 'kmeans'):
        self.incremental_data_map = {}
        self.data  = np.array([])
        self.label = np.array([])
        self.k_value = prototype_nums
        # 2024 06 07
        self.cluster_strategy = cluster_strategy
    # 把新获得的数据按不同类别划分
    def split_current_data_by_class(self):
        """把当前轮的数据按类别划分"""
        # 获取唯一的类别标签
        current_unique_labels = np.unique(self.label)
        # 按类别分割当前轮获得的数据
        data_by_class = {}  # map {"label":[the data belong to the label]}
        for current_data_label in current_unique_labels:
            indices = np.where(self.label == current_data_label)[0]
            data_by_class[current_data_label] = self.data[indices]
        return data_by_class, current_unique_labels
    # 将划分好的数据按不同类别和不同类别的原型数据进行拼接
    def data_combined(self):
        """把当前轮的数据，和类map中的数据进行合并"""
        current_data_map_by_class, current_unique_labels = self.split_current_data_by_class()
        # 考虑到有新类出现的情况
        # 如果current_unique_labels有新label,直接扩充incremental_data_map
        # 如果label 是incremental_data_map中已经有的，扩张incremental_data_map对应label中data的长度
        for new_data_label in current_unique_labels:
            if new_data_label in self.incremental_data_map:
                # 增量map中已经有这个标签的数据了，那就扩充这个数据
                self.incremental_data_map[new_data_label] = np.concatenate(
                    (self.incremental_data_map[new_data_label], current_data_map_by_class[new_data_label])
                )
            else:
                # 如果增量map中没有这个标签的的数据，就扩充增量map
                self.incremental_data_map[new_data_label] = current_data_map_by_class[new_data_label]
    def cluster_data(self,data, num_clusters):
        # ['kmeans','spectral','kmeans++','kmedoids','OPTICS','meanshift','gmm']
        # kmeans = KMeans(n_clusters=num_clusters,n_init='auto')
        cluster_centers = np.ndarray([])
        if self.cluster_strategy == 'kmeans':
            with warnings.catch_warnings():
            # KMeans(n_clusters=num_clusters,n_init='auto')
                kmeans = KMeans(n_clusters=num_clusters,n_init='auto')
                kmeans.fit(data)
                cluster_centers = kmeans.cluster_centers_
        elif self.cluster_strategy == 'spectral':
            with warnings.catch_warnings():
                # 报错，小类样本太少，无法聚类
                print('cluster : spectral',{num_clusters})
                spectral = SpectralClustering(n_clusters=num_clusters, random_state=0, affinity='nearest_neighbors').fit(data)
                labels_spectral = spectral.labels_
                cluster_centers = np.array([data[labels_spectral == i].mean(axis=0) for i in range(num_clusters)])
                print(len(labels_spectral))
                print(len(cluster_centers))
        elif self.cluster_strategy.lower() == 'hdbscan':
            with warnings.catch_warnings():
                min_cluster_size = 2
                hdb = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size).fit(data)
                labels_hdbscan = hdb.labels_
                cluster_centers = np.array([data[labels_hdbscan == i].mean(axis=0) for i in range(len(set(labels_hdbscan)) - (1 if -1 in labels_hdbscan else 0))])
        elif self.cluster_strategy.lower() == 'kmeans++' or self.cluster_strategy.lower() == 'kmeansplusplus' :
            with warnings.catch_warnings():
                kmeans_plus_plus = KMeans(n_clusters=num_clusters, init='k-means++', random_state=0,n_init='auto').fit(data)
                cluster_centers = kmeans_plus_plus.cluster_centers_
        elif self.cluster_strategy.lower() =='kmedoids':
            with warnings.catch_warnings():
                # 有几个空簇
                print('cluster : spectral',{num_clusters})
                kmedoids = KMedoids(n_clusters=num_clusters-10, random_state=0).fit(data)
                labelxx = kmedoids.labels_
                cluster_centers = kmedoids.cluster_centers_
                print(len(cluster_centers))
        elif self.cluster_strategy.upper() =='OPTICS':
            with warnings.catch_warnings():
                min_samples = 2
                optics = OPTICS(min_samples=min_samples).fit(data)
                labels_optics = optics.labels_
                cluster_centers = np.array([data[labels_optics == i].mean(axis=0) for i in range(len(set(labels_optics)) - (1 if -1 in labels_optics else 0))])
                print(len(cluster_centers))
        elif self.cluster_strategy.lower() =='meanshift':
            with warnings.catch_warnings():
                bandwidth = 0.1
                mean_shift = MeanShift(bandwidth=bandwidth).fit(data)
                cluster_centers = mean_shift.cluster_centers_
        elif self.cluster_strategy.lower() =='gmm':
            with warnings.catch_warnings():
                gmm = GaussianMixture(n_components=num_clusters, random_state=0).fit(data)
                cluster_centers = gmm.means_  # 高斯混合模型的质心是每个成分的均值
        return  cluster_centers
    def reCludter(self):
        # incremental_data_map的大小进行压缩self.reCluster_instance_nums
        new_cluster = {}
        for label, data in self.incremental_data_map.items():
            if len(data) <= self.k_value:
                # 全部保存
                new_cluster[label] = data
            else:
                sampled_nums = self.k_value
                clusters = self.cluster_data(data = data, num_clusters=sampled_nums)
                new_cluster[label] = clusters
        self.incremental_data_map = new_cluster
    def compute_sampling_nums(self,combinde_map):
        # initialize
        min_length = float('inf')  # 初始化最小长度为正无穷大
        max_length = 0  # 初始化最大长度为0
        for label, data in combinde_map.items():
            data_length = len(data)
            if data_length < min_length:
                min_length = data_length
            if data_length > max_length:
                max_length = data_length
        return min_length, max_length
    def data_sampling(self,sampling_nums):
        resampling_data = []
        resampling_label = []
        for label, data in self.incremental_data_map.items():
            if len(data) > sampling_nums:
                # 那就是下采样了,不放回采样
                sampled_indices = random.sample(range(len(data)), sampling_nums)
                sampled_data = [data[i] for i in sampled_indices]
                resampling_data.extend(sampled_data)
                resampling_label.extend([label] * sampling_nums)
            elif len(data) == sampling_nums:
                # 直接复制
                resampling_data.extend(data)
                resampling_label.extend([label] * sampling_nums)
            else:
                # 上采样,保存原样本
                resampling_data.extend(data)
                # 随机有放回的找差额部分
                sampled_data = random.choices(data, k=(sampling_nums-len(data)))
                resampling_data.extend(sampled_data)
                resampling_label.extend([label] * sampling_nums)
        # 洗牌
        combined_data = list(zip(resampling_data, resampling_label))
        random.shuffle(combined_data)
        resampling_data, resampling_label = zip(*combined_data)
        return resampling_data,resampling_label
    def fit(self,incremental_prototype_map,data,label,sampling_strategy = "OverSampling"):
        self.incremental_data_map = incremental_prototype_map
        self.data = data
        self.label = label
        self.data_combined()
        resampling_data = []
        resampling_label = []
        min_length, max_length =self.compute_sampling_nums(self.incremental_data_map)
        if sampling_strategy.lower() == "oversampling":
            resampling_data,resampling_label = self.data_sampling(max_length)
        elif sampling_strategy.lower() == "downsampling":
            resampling_data,resampling_label = self.data_sampling(min_length)
        else:
            print("No sampling measures have been taken")
        self.reCludter()
        return resampling_data,resampling_label,self.incremental_data_map
# ['kmeans','spectral','kmeans++','kmedoids','OPTICS','meanshift','gmm']

class Triplets(object):
    def __init__(self, n_neighbors=5, random=True, len_lim=True, **kwargs):
        self.n_neighbors = n_neighbors
        self.random = random
        self.len_lim = len_lim

    def fit_resample(self, x, y):
        strategy = self._sample_strategy(y)
        self.n_neighbors = max(self.n_neighbors, self.counts.max() // self.counts.min())

        gen_x = []
        gen_y = []
        # 这里的代码平衡状态会报错
        for c, size in enumerate(strategy):
            if size == 0: continue
            weight = self._weights(x, y, c)
            gen_x_c, gen_y_c = self._sample_one(x, y, c, size, weight)
            gen_x += gen_x_c
            gen_y += gen_y_c

        # 为了这个方法在平衡状态下不报错，我们特地在这里加了这段代码
        # To prevent errors in this method when in a balanced state, we intentionally added this code block
        if len(gen_x)==0:
            return x,y
        gen_x = np.vstack(gen_x)
        gen_y = np.array(gen_y)
        return np.concatenate((x, gen_x), axis=0), np.concatenate((y, gen_y), axis=0)

    def _sample_strategy(self, y):
        _, self.counts = np.unique(y, return_counts=True)
        return max(self.counts) - self.counts

    def _weights(self, x, y, c):
        return np.ones(self.counts[c])

    def _sample_one(self, x, y, c, size, weight):
        gen_x = []
        gen_y = []
        if size == 0: return gen_x, gen_y

        # get the indices of minority and majority instances
        min_idxs = np.where(y == c)[0]
        maj_idxs = np.where(y != c)[0]

        # find nearest majority neighbors for each minority instance
        nbrs = NearestNeighbors(n_neighbors=self.n_neighbors).fit(x[maj_idxs])
        _, indices = nbrs.kneighbors(x[min_idxs])

        # generate synthetic data
        for j in np.random.choice(len(min_idxs), size, p=weight / weight.sum()):
            tp1 = x[min_idxs[j]]
            tp2 = x[maj_idxs[indices[j][:5]]].mean(axis=0)
            # tp3_ord = np.random.randint(1, self.n_neighbors)
            tp3_ord = np.random.randint(self.n_neighbors)
            tp3 = x[maj_idxs[indices[j][tp3_ord]]]
            if (tp2 == tp1).all():
                gen_x.append(tp1)
                gen_y.append(c)
                continue

            offset = tp3 - tp2
            if self.len_lim: offset = offset * min(1, norm(tp1 - tp2) / norm(offset))
            coef = np.random.rand() if self.random is True else 1.0
            new_x = tp1 + coef * offset
            gen_x.append(new_x)
            gen_y.append(c)
        return gen_x, gen_y

def train_model_FedAvg_local(input_model,X_train_tensor, y_train_tensor, num_epochs):
    losses =[]
    model = input_model# copy.deepcopy(input_model)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
        losses.append(loss.item())  # 将每次训练的损失值添加到列表中

    return losses
def train_model_FedProx_local(input_model, X_train_tensor, y_train_tensor, num_epochs):
    mu = 0.1
    # because last round trained global model replaced local model,
    # that in this round the first local model is last round global model
    losses =[]
    model = input_model# copy.deepcopy(input_model)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    global_weights = copy.deepcopy(list(model.parameters()))
    # not use deepcopy ,because model as parameter transport in this ,update model also update model
    # current_local_model = cmodel
    # model.train()
    # for epoch in range(num_epochs):
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        # FedProx
        prox_term = 0.0
        for p_i, param in enumerate(model.parameters()):
                prox_term += (mu / 2) * torch.norm((param - global_weights[p_i])) ** 2
        loss += prox_term

        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
        losses.append(loss.item())  # 将每次训练的损失值添加到列表中
    #     total_loss = 0.0
    #     for step,(x,y) in enumerate(zip(X_train_tensor,y_train_tensor)):
    #         # current_local_model.train()
    #         output = model(x) # current_local_model(x)
    #         loss = criterion(output, y)
    #         total_loss+=loss.item()
    #         optimizer.zero_grad()
    #
    #         # fedprox
    #         prox_term = 0.0
    #         for p_i, param in enumerate(model.parameters()):
    #             prox_term += (mu / 2) * torch.norm((param - global_weights[p_i])) ** 2
    #         loss += prox_term
    #         loss.backward()
    #         optimizer.step()
    #     losses.append(total_loss)
    #     if epoch % 10 == 0:
    #         print(f"Epoch [{epoch+1}/{num_epochs}], Epoch Total Loss: {total_loss:.4f}")
    return losses

def train_model_FedNova_local(input_model,X_train_tensor, y_train_tensor, num_epochs):
    # because last round trained global model replaced local model,
    # that in this round the first local model is last round global model
    losses =[]
    model = input_model
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    global_weights = copy.deepcopy(input_model.state_dict())
    tau = 0
    rho = 0.9
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        tau +=len(y_train_tensor)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
        losses.append(loss.item())
    # for epoch in range(num_epochs):
    #     total_loss = 0.0
    #     for step,(x,y) in enumerate(zip(X_train_tensor,y_train_tensor)):
    #         # current_local_model.train()
    #         # model.train()
    #         output = model(x) # current_local_model(x)
    #         loss = criterion(output, y)
    #         total_loss+=loss.item()
    #         optimizer.zero_grad()
    #         loss.backward()
    #         optimizer.step()
    #         tau +=1
    #     losses.append(total_loss)
    #     if epoch % 10 == 0:
    #         print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")
    coeff = (tau - rho * (1 - pow(rho, tau)) / (1 - rho)) / (1 - rho)
    state_dict = model.state_dict()
    norm_grad = copy.deepcopy(global_weights)
    for key in norm_grad:
        norm_grad[key] = torch.div(global_weights[key] - state_dict[key], coeff)

    return losses, coeff, norm_grad,len(X_train_tensor)
# 定义模型参数共享函数
def share_params(model):
    params = model.state_dict()
    # return {k: v.clone().detach().requires_grad_(True) for k, v in params.items()}
    return {k: v.clone().detach().requires_grad_(False) for k, v in params.items()}

# # 定义模型参数聚合函数
def aggregate_params(params_list):
    aggregated_params = {}
    for key in params_list[0].keys():
        # 将参数转换为张量进行处理
        params_tensors = [params[key].clone().detach().float() for params in params_list]
        # 聚合参数
        aggregated_params[key] = sum(params_tensors) / len(params_tensors)
    return aggregated_params
def test(global_model,X_test,y_test):
    # 在全局模型上进行测试
    with torch.no_grad():
        X_test_tensor = torch.FloatTensor(X_test)
        y_test_tensor = torch.LongTensor(y_test)
        outputs = global_model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
        print(f"Test Accuracy: {accuracy * 100:.2f}%")

        # 计算度量
        predictions = predicted.numpy() # 将张量转换为 NumPy 数组并去除零维数组
        true_labels = y_test_tensor.numpy()  # 将张量转换为 NumPy 数组并去除零维数组
        precision = precision_score(true_labels,predictions,zero_division=0.0,average=None)
        precision_micro = precision_score(true_labels,predictions,zero_division=0.0,average='micro')
        precision_macro = precision_score(true_labels,predictions,zero_division=0.0,average='macro')
        # recall
        recalls = recall_score(true_labels,predictions,zero_division=0.0,average=None)
        recalls_micro =recall_score(true_labels,predictions,zero_division=0.0,average='micro')
        recalls_macro =recall_score(true_labels,predictions,zero_division=0.0,average='macro')
        f1_scores = f1_score(true_labels, predictions, average=None)
        acc = accuracy_score(true_labels, predictions)
        kappa = cohen_kappa_score(true_labels,predictions)
        conf_matrix = confusion_matrix(true_labels,predictions)
        # 计算所有类别乘积的几何平均值作为 G-mean
        g_mean_all= np.power(np.prod(recalls), 1 / len(recalls))
        # AUC
        lb = LabelBinarizer()
        lb.fit(true_labels)
        true_labels_bin = lb.transform(true_labels)
        predictions_bin = lb.transform(predictions)
        auc = roc_auc_score(true_labels_bin, predictions_bin, average='weighted', multi_class='ovr')
        metrics = {
            'recall':recalls,
            'recall_micro':recalls_micro,
            'recall_macro':recalls_macro,
            'precision':precision,
            'precision_micro':precision_micro,
            'precision_macro':precision_macro,
            'f1_score':f1_scores,
            'g_mean':g_mean_all,
            'acc':acc,
            'auc':auc,
            'kappa':kappa,
            'confusion_matrix':conf_matrix
        }
        return metrics

def save_loss(loss_list,client_id,round_id,save_loss_path):
    if not os.path.exists(save_loss_path):
        os.makedirs(save_loss_path)
    # 构建文件路径
    file_path = os.path.join(save_loss_path, f"client_{client_id}.csv")

    if os.path.exists(file_path):
        # 如果文件存在，加载现有的 CSV 文件为 DataFrame
        df = pd.read_csv(file_path)
    else:
        # 如果文件不存在，直接创建新的 DataFrame
        df = pd.DataFrame()

    # 将损失值添加到 DataFrame 中
    column_name = f'round_{round_id}'
    df[column_name] = loss_list

    # 将 DataFrame 保存为 CSV 文件
    df.to_csv(file_path, index=False)
def save_model(global_model,round_id,save_model_path):
    if not os.path.exists(save_model_path):
        os.makedirs(save_model_path)
    model_path = os.path.join(save_model_path,f'round_{round_id}_gm.pt')
    torch.save(global_model,model_path)

def save_metrics(title, rounds, metrics, save_folder):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    file_name = f"{title}.csv"
    file_path = os.path.join(save_folder, file_name)
    recalls = metrics['recall']
    class_nums = len(recalls)
    # 检查文件是否存在
    if os.path.exists(file_path):
        # 如果文件存在，加载现有的 Excel 文件为 DataFrame
        df = pd.read_csv(file_path)
    else:
        # 如果文件不存在，直接创建新的 DataFrame
        columns = [
        'rounds', 'accuracy', 'auc', 'kappa', 'g_mean', 'recall_micro', 'precision_micro',
        'recall_macro', 'precision_macro'
        ]
        df = pd.DataFrame(columns=columns)
        for i in range(class_nums):  # 动态生成 f1-score 相关列名
            columns.append(f'f1_score_{i}')
            columns.append(f'recall_{i}')
            columns.append(f'precession_{i}')

    data = {
        'rounds': rounds,
        'accuracy': metrics['acc'],
        'auc': metrics['auc'],
        'kappa': metrics['kappa'],
        'g_mean':metrics['g_mean'],
        'recall_micro':metrics['recall_micro'],
        'precision_micro':metrics['precision_micro'],
        'recall_macro':metrics['recall_macro'],
        'precision_macro':metrics['precision_macro']
    }
    # 添加每个类别的 F1-score、G-mean 和 Recall 到 data 中
    for i in range(class_nums):  #类别数
        data[f'recall_{i}'] = metrics['recall'][i]
        data[f'precision_{i}'] = metrics['precision'][i]
        data[f'f1_score_{i}'] = metrics['f1_score'][i]
    # 创建新行并追加到 DataFrame
    new_row = pd.DataFrame(data, index=[0])
    df = pd.concat([df, new_row], ignore_index=True)

    # 将 DataFrame 保存为 Excel 文件
    df.to_csv(file_path, index=False)

from imblearn.over_sampling import SMOTE,RandomOverSampler,SMOTENC,SMOTEN,ADASYN,BorderlineSMOTE,KMeansSMOTE,SVMSMOTE
from imblearn.under_sampling import ClusterCentroids,RandomUnderSampler,NearMiss,TomekLinks,EditedNearestNeighbours,RepeatedEditedNearestNeighbours,AllKNN,CondensedNearestNeighbour,OneSidedSelection,NeighbourhoodCleaningRule,InstanceHardnessThreshold
from imblearn.combine import SMOTEENN,SMOTETomek
# SMOTE,ROS,SMOTENC,SMOTEN,ADASYN,BorderlineSMOTE1,BorderlineSMOTE2,KMeansSMOTE,SVMSMOTE
# ClusterCentroids,RUS,NearMiss1,NearMiss2,NearMiss2,TomekLinks,ENN,RENN,AllKNN,CNN,OSS,NC,IHT
# SMOTEENN,SMOTETomek
def data_sampling(raw_X,raw_y,sampling_strategy):
    if sampling_strategy.upper() == 'NO' :
        return raw_X,raw_y
    # overSampling
    elif sampling_strategy.upper() == 'SMOTE': # overSampling
        """
            1.对于样本x ,按照欧氏距离找到离其距离最近的K个近邻样本
            2.确定采样比例，然后从K个近邻中选择x_n
            3.公式 x_new = x + rand(0,1)*(x_n-x)
        """
        smote = SMOTE( random_state=42)
        X_resampled, y_resampled = smote.fit_resample(raw_X,raw_y)
        return X_resampled, y_resampled
    elif sampling_strategy == "RandomOverSampler" or sampling_strategy.upper()=='ROS': # overSampling
        ros = RandomOverSampler(random_state=1)
        ros_data,ros_label = ros.fit_resample(raw_X,raw_y)
        return ros_data,ros_label
    elif sampling_strategy.upper() == 'SMOTENC':    # overSampling
        smotenc = SMOTENC(random_state=1,categorical_features=[0])
        smotenc_data,smotenc_label = smotenc.fit_resample(raw_X,raw_y)
        return smotenc_data,smotenc_label
    elif sampling_strategy.upper() == 'SMOTEN': # overSampling
        smoten = SMOTEN(random_state=1)
        smoten_data,smoten_label = smoten.fit_resample(raw_X,raw_y)
        return smoten_data,smoten_label
    elif sampling_strategy.upper() =='ADASYN':
        adasyn = ADASYN(random_state=1)
        adasyn_data,adasyn_label = adasyn.fit_resample(raw_X,raw_y)
        return adasyn_data,adasyn_label
    elif sampling_strategy == 'BorderlineSMOTE1' or sampling_strategy.upper()=='BSMOTE1':
        bsmote1 = BorderlineSMOTE(kind='borderline-1',random_state=1)
        bsmote1_data,bsmote1_label = bsmote1.fit_resample(raw_X,raw_y)
        return bsmote1_data,bsmote1_label
    elif sampling_strategy == 'BorderlineSMOTE2'or sampling_strategy.upper()=='BSMOTE2':
        bsmote2 = BorderlineSMOTE(kind='borderline-2',random_state=1)
        bsmote2_data,bsmote2_label = bsmote2.fit_resample(raw_X,raw_y)
        return bsmote2_data,bsmote2_label
    elif sampling_strategy == 'KMeansSMOTE' or sampling_strategy.upper() == 'KSMOTE':
        kmeanssmote = KMeansSMOTE(random_state=1)
        kmeanssmote_data,kmeanssmote_label = kmeanssmote.fit_resample(raw_X,raw_y)
        return kmeanssmote_data,kmeanssmote_label
    elif sampling_strategy == 'SVMSMOTE':
        svmsmote = SVMSMOTE(random_state=1)
        svmsmote_data,svmsmote_label = svmsmote.fit_resample(raw_X,raw_y)
        return svmsmote_data,svmsmote_label
    # downSampling
    elif sampling_strategy == 'ClusterCentroids': # down-sampling,generate
        clustercentroids = ClusterCentroids(random_state=1)
        clustercentroids_data,clustercentroids_label = clustercentroids.fit_resample(raw_X,raw_y)
        return clustercentroids_data,clustercentroids_label
    elif sampling_strategy=='RandomUnderSampler' or sampling_strategy.upper()=='RUS':
        rus = RandomUnderSampler(random_state=1)
        rus_data,rus_label = rus.fit_resample(raw_X,raw_y)
        return rus_data,rus_label
    elif sampling_strategy.upper() =='NEARMISS1':
        # 在k个少数类别样本中，选择出与他们-平均距离最近的多数类样本-进行保存
        nearmiss1 = NearMiss(version=1)
        nearmiss1_data,nearmiss1_label = nearmiss1.fit_resample(raw_X,raw_y)
        return nearmiss1_data,nearmiss1_label
    elif sampling_strategy.upper() =='NEARMISS2':
        # 选择K个距离最远的少数类别样本，然后根据这些样本选出的"平均距离最近"的样本进行保存
        nearmiss2 = NearMiss(version=2)
        nearmiss2_data,nearmiss2_label = nearmiss2.fit_resample(raw_X,raw_y)
        return nearmiss2_data,nearmiss2_label
    elif sampling_strategy.upper() =='NEARMISS3':
        # 1、对于每一个少数类别样本，保留其K个最近邻多数类样本；2、把到K个少数样本平均距离最大的多数类样本保存下来。
        nearmiss3 = NearMiss(version=3)
        nearmiss3_data,nearmiss3_label = nearmiss3.fit_resample(raw_X,raw_y)
        return nearmiss3_data,nearmiss3_label
    elif sampling_strategy == 'TomekLinks' or sampling_strategy.upper()=='TOMEK':
        # 它需要计算每个样本之间的距离，然后把互为最近邻且类别不同的一对样本拿出来，根据需求的选择将这一对数据进行剔除 or 把多数类样本剔除
        tomelink = TomekLinks(sampling_strategy='all')#sampling_strategy='all'表示全部删除，'auto'表示只删除多数类
        tomelink_data,tomelink_label = tomelink.fit_resample(raw_X,raw_y)
        return tomelink_data,tomelink_label
    elif sampling_strategy == 'EditedNearestNeighbours' or sampling_strategy.upper() == 'ENN':
        ENN = EditedNearestNeighbours()
        ENN_data,ENN_label = ENN.fit_resample(raw_X,raw_y)
        return ENN_data,ENN_label
    elif sampling_strategy == 'RepeatedEditedNearestNeighbours' or sampling_strategy.upper() == 'RENN':
        RENN = RepeatedEditedNearestNeighbours()
        RENN_data,RENN_label = RENN.fit_resample(raw_X,raw_y)
        return RENN_data,RENN_label
    elif sampling_strategy =='AllKNN':
        ## ENN的改进版本，和RepeatedEditedNearestNeighbours一样，会多次迭代ENN 算法，不同之处在于，他会每次增加KNN的K值
        allknn = AllKNN()
        allknn_data,allknn_label = allknn.fit_resample(raw_X,raw_y)
        return allknn_data,allknn_label
    elif sampling_strategy == 'CondensedNearestNeighbour'or sampling_strategy.upper() == 'CNN':
        ## 如果有样本无法和其他多数类样本聚类到一起，那么说明它极有可能是边界的样本，所以将这些样本加入到集合中
        CNN = CondensedNearestNeighbour(random_state=1)
        CNN_data,CNN_label = CNN.fit_resample(raw_X,raw_y)
        return CNN_data,CNN_label
    elif sampling_strategy == 'OneSidedSelection' or sampling_strategy.upper() == 'OSS':
        # OneSidedSelection = tomekLinks + CondensedNearestNeighbour,先使用自杀式的方式把大类数据中的其他值剔除，然后再使用CondensedNearestNeighbour的下采样
        OSS = OneSidedSelection(random_state=1)
        OSS_data,OSS_label = OSS.fit_resample(raw_X,raw_y)
        return OSS_data,OSS_label
    elif sampling_strategy == 'NeighbourhoodCleaningRule'or sampling_strategy.upper() == 'NC':
        # 若在大类的K-近邻中，少数类占多数，那就剔除这个多数类别的样本
        NC = NeighbourhoodCleaningRule()
        NC_data,NC_label = NC.fit_resample(raw_X,raw_y)
        return NC_data,NC_label
    elif sampling_strategy == 'InstanceHardnessThreshold' or sampling_strategy.upper() == 'IHT':
        # 默认算法是随机森林，通过分类算法给出样本阈值来剔除部分样本，（阈值较低的可以剔除）,慢
        IHT = InstanceHardnessThreshold(random_state=1)
        IHT_data,IHT_label = IHT.fit_resample(raw_X,raw_y)
        return IHT_data,IHT_label
    # hibird
    elif sampling_strategy.upper() =='SMOTEENN':
        se = SMOTEENN(random_state=1)
        se_data,se_label = se.fit_resample(raw_X,raw_y)
        return se_data,se_label
    elif sampling_strategy.upper() =='SMOTETOMEK':
        st = SMOTETomek(random_state=1)
        st_data,st_label = st.fit_resample(raw_X,raw_y)
        return st_data,st_label
    elif sampling_strategy == 'Triplets':
        print(" Triplets sampling")
        tpl = Triplets()
        tpl_data,tpl_label = tpl.fit_resample(raw_X,raw_y)
        return tpl_data,tpl_label
    else :
        print("skipped all the sampling strategy,but return the raw data and label")
        return raw_X,raw_y
# ['kmeans','spectral','kmeans++','kmedoids','OPTICS','meanshift','gmm']
def inremental_sampling(prototype_map,raw_X,raw_y,cluster_strategy):
    class_nums = len(np.unique(raw_y))
    print("class nums" ,class_nums)
    isap = IncrementalSampling(cluster_strategy=cluster_strategy)
    resampling_data,resampling_label,prototype_map = isap.fit(incremental_prototype_map=prototype_map,data=raw_X,label=raw_y)
    return resampling_data,resampling_label,prototype_map
# ['kmeans','spectral','kmeans++','kmedoids','OPTICS','meanshift','gmm']
def read_data_return_tensor(dataset_path, round_id, client_id, sampling_strategy='no',prototype_map = {},cluster_strategy='kmeans'):
    folder_path = os.path.join(dataset_path, f'client_{client_id}')
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    open_file_path = os.path.join(folder_path, f'round_{round_id}.csv')
    data = pd.read_csv(open_file_path, header=0)
    # 排除标题行并打乱数据
    data_shuffled = shuffle(data)  # 排除标题行并打乱数据

    # 提取特征和目标变量
    raw_X = data_shuffled.iloc[:, :-1].values  # 特征
    raw_y = data_shuffled.iloc[:, -1].values   # 目标变量
    raw_y = raw_y.astype(float)
    # print(len(raw_X),type(raw_X))
    # print(len(raw_X),type(raw_X))
    if sampling_strategy == 'IncrementalSampling':
        print("using IncrementalSampling")
        resampling_X,resampling_y,prototype_map = inremental_sampling(prototype_map,raw_X,raw_y,cluster_strategy=cluster_strategy)
    else:
        resampling_X,resampling_y = data_sampling(raw_X,raw_y,sampling_strategy)
        resampling_y.astype(float)
    resampling_X = np.array(resampling_X)  # 将列表转换为单个NumPy数组
    resampling_y = np.array(resampling_y)  # 将列表转换为单个NumPy数组

    X_train_tensor = torch.tensor(resampling_X, dtype=torch.float32)  # 特征张量
    y_train_tensor = torch.tensor(resampling_y, dtype=torch.long)   # 标签张量
    return X_train_tensor,y_train_tensor,prototype_map
def read_test_data(test_data_path):
    data = pd.read_csv(test_data_path, header=0)
    # 排除标题行并打乱数据
    data_shuffled = shuffle(data)  # 排除标题行并打乱数据

    # 提取特征和目标变量
    X = data_shuffled.iloc[:, :-1].values.astype(float)  # 特征
    # print(X)
    y = data_shuffled.iloc[:, -1].values   # 目标变量
    y = y.astype(float)

    X_test_tensor = torch.tensor(X, dtype=torch.float32)  # 特征张量
    y_test_tensor = torch.tensor(y, dtype=torch.long)   # 标签张量
    return X_test_tensor,y_test_tensor


In [ ]:
def aggregate_fednova(local_params_list,gm):
    # (share_params(clients_models[i]),coeff, norm_grad, data_len) as input
    total_data_len = sum(data_len for _, _, _, data_len in local_params_list)
    global_model_state = gm.state_dict()
    nova_model_state = copy.deepcopy(global_model_state)
    # avg_loss = 0
    coeff = 0.0
    for clientID,(client_model,client_coeff,client_norm_grad,client_local_data_len) in enumerate(local_params_list):
        coeff = coeff + client_coeff*client_local_data_len/total_data_len
        for key in client_model.state_dict():
            if clientID == 0:
                nova_model_state[key] = client_norm_grad[key] * client_local_data_len/total_data_len
            else:
                nova_model_state[key] =nova_model_state[key]+ client_norm_grad[key] * client_local_data_len/total_data_len
        # avg_loss = avg_loss + cl
    for key in global_model_state:
        global_model_state[key] -= coeff*nova_model_state[key]

    return global_model_state
pp = []
def runFedNova(samplingName,settingName,cluster_strategy):
    num_clients = 10
    # 初始化全局模型和客户端模型
    input_size = 10
    hidden_size = 100
    output_size = 5
    global_model = MLP(input_size, hidden_size, output_size)
    client_prototype_map = [{} for _ in range(num_clients)]
    clients_models = [MLP(input_size, hidden_size, output_size) for _ in range(num_clients)]

    num_epochs = 200
    num_global_updates = 195
    # 'E:/FedStream/real_data_set/realdataset0427/elecNorm/Electricity_client_random/'
    base_path ='E:/FedStream/' # 'E:/FedStream/'
    dataset_name = 'pokerhand_five'
    setting_name = settingName # 'CovA_Abrupt_BtoVH'
    # SMOTE cant use in VH,H
    # sampling_strategy_name_list = ['no','RandomOverSampler','RandomUnderSampler','CondensedNearestNeighbour','Triplets','IncrementalSampling']
    # sampling_strategy_name = sampling_strategy_name_list[sampling_id]
    sampling_strategy_name = samplingName

    algorithm = 'FedNova_kvalue30'
    experiment_times = 'epoch200'
    save_loss_path = f'{base_path}/loss/{dataset_name}_Sampling/{algorithm}/{setting_name}/{sampling_strategy_name}_{cluster_strategy}_{experiment_times}'
    save_model_path = f'{base_path}/models/{dataset_name}_Sampling/{algorithm}/{setting_name}/{sampling_strategy_name}_{cluster_strategy}_{experiment_times}'
    save_metrics_path  = f'{base_path}/metrics/{dataset_name}_Sampling/{algorithm}/{setting_name}/{sampling_strategy_name}_{cluster_strategy}_{experiment_times}'
    # E:\FedStream\real_data_set\realdataset0427\covertypeNorm\CoverType_client_Full_class
    read_data_path = f'E:/FedStream/real_data_set/realdataset0427/{dataset_name}/{setting_name}/'
    # 读取测试集CSV文件并转换为PyTorch张量
    # E:\FedStream\real_data_set\realdataset0427\covertypeNorm
    test_path = 'E:/FedStream/real_data_set/realdataset0427/pokerhand_five/test.csv'
    X_test_tensor,y_test_tensor = read_test_data(test_path)

    for update in range(num_global_updates):
        print(f"round{update}")
        local_params_list = []

        for i in range(num_clients):
            # 在每个客户端训练本地模型
            print(f"client{i}")
            X_train_local,y_train_local,prototype_map_r = read_data_return_tensor(read_data_path,round_id=update,client_id=i,sampling_strategy=sampling_strategy_name,prototype_map=client_prototype_map[i],cluster_strategy=cluster_strategy)
            client_prototype_map[i] = prototype_map_r
            # 训练本地模型并获取损失值
            losses, coeff, norm_grad, data_len = train_model_FedNova_local(clients_models[i],X_train_local,y_train_local,num_epochs=num_epochs)
            # y_train_tensor = torch.sub(y_train_tensor, 1)
            # losses, coeff, norm_grad, data_len = train_model_FedNova_local(clients_models[i],X_train_local,torch.sub(y_train_local, 1),num_epochs=num_epochs)
            # save_loss(loss_list=losses,client_id=i,round_id=update,save_loss_path=save_loss_path)

            local_metrics = test(copy.deepcopy(clients_models[i]),X_test_tensor,y_test_tensor)
            local_params_list.append((copy.deepcopy(clients_models[i]),coeff, norm_grad, data_len))
            save_metrics(title=f"client_{i}_metrics", rounds=update, metrics=local_metrics,save_folder = save_metrics_path)

        aggregated_params = aggregate_fednova(local_params_list,gm = copy.deepcopy(global_model))
        global_model.load_state_dict(aggregated_params)

        # 在每轮结束后发送全局模型参数给客户端
        gm = copy.deepcopy(global_model)
        save_model(copy.deepcopy(gm),update,save_model_path)
        for client_model in clients_models:
            client_model.load_state_dict(gm.state_dict())

        me = test(gm,X_test_tensor,y_test_tensor) # torch.sub(y_train_tensor, 1)
        # me = test(gm,X_test_tensor,torch.sub(y_test_tensor, 1))
        save_metrics(title="global_back", rounds=update, metrics=me,save_folder = save_metrics_path)
        print("gme acc:" ,me)
import time

sampling_strategy_name_list =['IncrementalSampling'] # ['no','IncrementalSampling','RandomOverSampler', 'RandomUnderSampler', 'CondensedNearestNeighbour', 'Triplets']#, 'RandomOverSampler', 'RandomUnderSampler', 'CondensedNearestNeighbour', 'Triplets', 'IncrementalSampling']
dataset_list = ['pokerhand_client','pokerhand_client','pokerhand_client','pokerhand_client','pokerhand_client']#,'CoverType_client_Full_class','CoverType_client_Full_class','CoverType_client_Full_class','CoverType_client_Full_class']
cluster_strategy_list = ['gmm','kmeans++','OPTICS','meanshift'] # 'kmeans','kmeans++','OPTICS','meanshift','spectral','kmedoids',
total_start_time = time.time()  # 记录整个循环的开始时间
ex_time_list = []
for j, settingname in enumerate(dataset_list):
    for i, cluster_strategy in enumerate(cluster_strategy_list):
        start_time = time.time()  # 记录当前迭代的开始时间
        runFedNova(samplingName='IncrementalSampling', settingName=settingname,cluster_strategy=cluster_strategy)
        end_time = time.time()  # 记录当前迭代的结束时间
        execution_time = end_time - start_time  # 计算当前迭代的执行时间
        ex_time_list.append(execution_time)
        time.sleep(20)

total_end_time = time.time()  # 记录整个循环的结束时间
total_execution_time = total_end_time - total_start_time  # 计算整个循环的执行时间

print(f"Total execution time: {total_execution_time} seconds")
print(ex_time_list)


round0
client0
using IncrementalSampling
class nums 5
Epoch [1/200], Loss: 1.9482
Epoch [11/200], Loss: 1.6062
Epoch [21/200], Loss: 1.5344
Epoch [31/200], Loss: 1.4803
Epoch [41/200], Loss: 1.4370
Epoch [51/200], Loss: 1.4015
Epoch [61/200], Loss: 1.3718
Epoch [71/200], Loss: 1.3463
Epoch [81/200], Loss: 1.3236
Epoch [91/200], Loss: 1.3027
Epoch [101/200], Loss: 1.2839
Epoch [111/200], Loss: 1.2666
Epoch [121/200], Loss: 1.2505
Epoch [131/200], Loss: 1.2352
Epoch [141/200], Loss: 1.2207
Epoch [151/200], Loss: 1.2067
Epoch [161/200], Loss: 1.1934
Epoch [171/200], Loss: 1.1817
Epoch [181/200], Loss: 1.1706
Epoch [191/200], Loss: 1.1600
Test Accuracy: 27.35%
client1
using IncrementalSampling
class nums 5
Epoch [1/200], Loss: 2.0749
Epoch [11/200], Loss: 1.5651
Epoch [21/200], Loss: 1.4897
Epoch [31/200], Loss: 1.4314
Epoch [41/200], Loss: 1.3837
Epoch [51/200], Loss: 1.3434
Epoch [61/200], Loss: 1.3079
Epoch [71/200], Loss: 1.2762
Epoch [81/200], Loss: 1.2478
Epoch [91/200], Loss: 1.2220